In [2]:
pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:

import math
import matplotlib
import numpy as np
import pandas as pd
import pickle
import plotly.graph_objs as go
import time

from collections import defaultdict
from tqdm.notebook import tqdm
from datetime import date
from matplotlib import pyplot as plt
from pylab import rcParams
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm_notebook
from xgboost import XGBRegressor
import pandas as pd # load and manipulate data and for one-hot encoding
import math
import numpy as np # calculate the mean and sd
import xgboost as xgb # xgboost stuff
from xgboost import XGBRegressor
from datetime import date
from collections import defaultdict
from sklearn.model_selection import train_test_split # split data intro training and testing sets
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer #for scoring
from sklearn.model_selection import GridSearchCV #cross validation
from sklearn.metrics import confusion_matrix # creates a confusion matrix
from sklearn.metrics import plot_confusion_matrix # draws a confusion matrix


In [4]:
xgb

<module 'xgboost' from '/home/km3ali/.local/lib/python3.7/site-packages/xgboost/__init__.py'>

In [5]:
cd stocks_50


[Errno 2] No such file or directory: 'stocks_50'
/mnt/storage/home/km3ali/stocks_50


In [6]:
cd stocks_50

[Errno 2] No such file or directory: 'stocks_50'
/mnt/storage/home/km3ali/stocks_50


In [7]:
df = pd.read_csv("TITAN.csv",error_bad_lines=False)

In [8]:
df.head()

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
0,2000-01-03,TITAN,EQ,144.95,146.0,156.45,146.0,155.00,155.70,154.36,23000,3.550370e+11,NaN,NaN,NaN
1,2000-01-04,TITAN,EQ,155.70,152.0,152.00,144.0,148.05,147.40,146.72,26300,3.858700e+11,NaN,NaN,NaN
2,2000-01-05,TITAN,EQ,147.40,144.0,148.80,136.0,139.95,138.40,142.95,20600,2.944850e+11,NaN,NaN,NaN
3,2000-01-06,TITAN,EQ,138.40,142.0,149.50,141.0,149.50,149.50,147.87,31600,4.672750e+11,NaN,NaN,NaN
4,2000-01-07,TITAN,EQ,149.50,149.5,153.00,145.0,147.75,146.35,148.63,36600,5.439965e+11,NaN,NaN,NaN


In [9]:
df.drop(['Trades','Deliverable Volume','%Deliverble'],axis = 1, inplace = True)


In [10]:
df.head()



,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover
0,2000-01-03,TITAN,EQ,144.95,146.0,156.45,146.0,155.00,155.70,154.36,23000,3.550370e+11
1,2000-01-04,TITAN,EQ,155.70,152.0,152.00,144.0,148.05,147.40,146.72,26300,3.858700e+11
2,2000-01-05,TITAN,EQ,147.40,144.0,148.80,136.0,139.95,138.40,142.95,20600,2.944850e+11
3,2000-01-06,TITAN,EQ,138.40,142.0,149.50,141.0,149.50,149.50,147.87,31600,4.672750e+11
4,2000-01-07,TITAN,EQ,149.50,149.5,153.00,145.0,147.75,146.35,148.63,36600,5.439965e+11


In [11]:
df['Symbol'].unique()

array(['TITAN'], dtype=object)

In [12]:
df['Series'].unique()


array(['EQ'], dtype=object)

In [13]:
df.drop(['Symbol','Series'],axis = 1, inplace = True)


In [14]:
df

,Date,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover
0,2000-01-03,144.95,146.0,156.45,146.00,155.00,155.70,154.36,23000,3.550370e+11
1,2000-01-04,155.70,152.0,152.00,144.00,148.05,147.40,146.72,26300,3.858700e+11
2,2000-01-05,147.40,144.0,148.80,136.00,139.95,138.40,142.95,20600,2.944850e+11
3,2000-01-06,138.40,142.0,149.50,141.00,149.50,149.50,147.87,31600,4.672750e+11
4,2000-01-07,149.50,149.5,153.00,145.00,147.75,146.35,148.63,36600,5.439965e+11
...,...,...,...,...,...,...,...,...,...,...
5282,2021-03-24,1495.35,1486.0,1508.80,1476.45,1485.00,1488.15,1492.58,1871092,2.792753e+14
5283,2021-03-25,1488.15,1484.1,1498.50,1447.75,1451.60,1450.85,1467.78,1609996,2.363123e+14
5284,2021-03-26,1450.85,1459.9,1512.40,1457.00,1505.50,1505.85,1490.91,1922278,2.865948e+14
5285,2021-03-30,1505.85,1510.0,1563.00,1510.00,1549.95,1552.20,1551.26,3466049,5.376747e+14


In [15]:
df

,Date,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover
0,2000-01-03,144.95,146.0,156.45,146.00,155.00,155.70,154.36,23000,3.550370e+11
1,2000-01-04,155.70,152.0,152.00,144.00,148.05,147.40,146.72,26300,3.858700e+11
2,2000-01-05,147.40,144.0,148.80,136.00,139.95,138.40,142.95,20600,2.944850e+11
3,2000-01-06,138.40,142.0,149.50,141.00,149.50,149.50,147.87,31600,4.672750e+11
4,2000-01-07,149.50,149.5,153.00,145.00,147.75,146.35,148.63,36600,5.439965e+11
...,...,...,...,...,...,...,...,...,...,...
5282,2021-03-24,1495.35,1486.0,1508.80,1476.45,1485.00,1488.15,1492.58,1871092,2.792753e+14
5283,2021-03-25,1488.15,1484.1,1498.50,1447.75,1451.60,1450.85,1467.78,1609996,2.363123e+14
5284,2021-03-26,1450.85,1459.9,1512.40,1457.00,1505.50,1505.85,1490.91,1922278,2.865948e+14
5285,2021-03-30,1505.85,1510.0,1563.00,1510.00,1549.95,1552.20,1551.26,3466049,5.376747e+14


In [16]:
df.sort_values(by = "Date",inplace = True, ascending = False)


In [17]:
def get_mape(y_true, y_pred): 
    """
    Compute mean absolute percentage error (MAPE)
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def get_mae(a, b):
    """
    Comp mean absolute error e_t = E[|a_t - b_t|]. a and b can be lists.
    Returns a vector of len = len(a) = len(b)
    """
    return np.mean(abs(np.array(a)-np.array(b)))

def get_rmse(a, b):
    """
    Comp RMSE. a and b can be lists.
    Returns a scalar.
    """
    return math.sqrt(np.mean((np.array(a)-np.array(b))**2))

In [18]:
def get_mov_avg_std(df, col, N):
    """
    Given a dataframe, get mean and std dev at timestep t using values from t-1, t-2, ..., t-N.
    Inputs
        df         : dataframe. Can be of any length.
        col        : name of the column you want to calculate mean and std dev
        N          : get mean and std dev at timestep t using values from t-1, t-2, ..., t-N
    Outputs
        df_out     : same as df but with additional column containing mean and std dev
    """
    mean_list = df[col].rolling(window = N, min_periods=1).mean() # len(mean_list) = len(df)
    std_list = df[col].rolling(window = N, min_periods=1).std()   # first value will be NaN, because normalized by N-1
    
    # Add one timestep to the predictions
    mean_list = np.concatenate((np.array([np.nan]), np.array(mean_list[:-1])))
    std_list = np.concatenate((np.array([np.nan]), np.array(std_list[:-1])))
    
    # Append mean_list to df
    df_out = df.copy()
    df_out[col + '_mean'] = mean_list
    df_out[col + '_std'] = std_list
    
    return df_out

def do_scaling(df, N):
    """
    Do scaling for the adj_close and lag cols
    """
    df.loc[:, 'Close_scaled'] = (df['Close'] - df['Close_mean']) / df['Close_std']
    for n in range(N,0,-1):
        df.loc[:, 'Close_scaled_lag_'+str(n)] = \
            (df['Close_lag_'+str(n)] - df['Close_mean']) / df['Close_std']
        
        # Remove adj_close_lag column which we don't need anymore
        df.drop(['Close_lag_'+str(n)], axis=1, inplace=True)

    return df

def pred_xgboost(model, N, H, prev_vals, prev_mean_val, prev_std_val):
    """
    Do recursive forecasting using xgboost
    Inputs
        model              : the xgboost model
        N                  : for feature at day t, we use lags from t-1, t-2, ..., t-N as features
        H                  : forecast horizon
        prev_vals          : numpy array. If predict at time t, 
                             prev_vals will contain the N unscaled values at t-1, t-2, ..., t-N
        prev_mean_val      : the mean of the unscaled values at t-1, t-2, ..., t-N
        prev_std_val       : the std deviation of the unscaled values at t-1, t-2, ..., t-N
    Outputs
        Times series of predictions. Numpy array of shape (H,). This is unscaled.
    """
    forecast = prev_vals.copy()

    for n in range(H):
        forecast_scaled = (forecast[-N:] - prev_mean_val) / prev_std_val
        
        # Create the features dataframe
        X = defaultdict(list)
        for n in range(N,0,-1):
            X['Close_scaled_lag_'+str(n)] = [forecast_scaled[-n]]
        X = pd.DataFrame(X)
             
        # Do prediction
        est_scaled = model.predict(X)
        
        # Unscale the prediction
        forecast = np.concatenate([forecast, 
                                   np.array((est_scaled * prev_std_val) + prev_mean_val).reshape(1,)])
        
        # Comp. new mean and std
        prev_mean_val = np.mean(forecast[-N:])
        prev_std_val = np.std(forecast[-N:])
           
    return forecast[-H:]

def train_pred_eval_model(X_train_scaled,
                          y_train_scaled,
                          y_test,
                          N,
                          H,
                          prev_vals,
                          prev_mean_val,
                          prev_std_val,
                          seed=100,
                          n_estimators=100,
                          max_depth=3,
                          learning_rate=0.1,
                          min_child_weight=1,
                          subsample=1,
                          colsample_bytree=1,
                          colsample_bylevel=1,
                          gamma=0):
    '''
    Train model, do prediction, scale back to original range and do evaluation
    Use XGBoost here.
    Inputs
        X_train_scaled     : features for training. Scaled to have mean 0 and variance 1
        y_train_scaled     : target for training. Scaled to have mean 0 and variance 1
        y_test             : target for test. Actual values, not scaled.
        N                  : for feature at day t, we use lags from t-1, t-2, ..., t-N as features
        H                  : forecast horizon
        prev_vals          : numpy array. If scaled[0] is at time t, prev_vals will contain the N-1 unscaled values at t-1, t-2, ...
        prev_mean_val      : the mean of the unscaled values at t-1, t-2, ..., t-N
        prev_std_val       : the std deviation of the unscaled values at t-1, t-2, ..., t-N
        seed               : model seed
        n_estimators       : number of boosted trees to fit
        max_depth          : maximum tree depth for base learners
        learning_rate      : boosting learning rate (xgb’s “eta”)
        min_child_weight   : minimum sum of instance weight(hessian) needed in a child
        subsample          : subsample ratio of the training instance
        colsample_bytree   : subsample ratio of columns when constructing each tree
        colsample_bylevel  : subsample ratio of columns for each split, in each level
        gamma              : 
    Outputs
        rmse               : root mean square error of y_test and est
        mape               : mean absolute percentage error of y_test and est
        mae                : mean absolute error of y_test and est
        est                : predicted values. Same length as y_test
    '''

    model = XGBRegressor(objective ='reg:squarederror',
                         seed=model_seed,
                         n_estimators=n_estimators,
                         max_depth=max_depth,
                         learning_rate=learning_rate,
                         min_child_weight=min_child_weight,
                         subsample=subsample,
                         colsample_bytree=colsample_bytree,
                         colsample_bylevel=colsample_bylevel,
                         gamma=gamma)
    
    # Train the model
    model.fit(X_train_scaled, y_train_scaled)
    
    # Get predicted labels and scale back to original range
    est = pred_xgboost(model, N, H, prev_vals, prev_mean_val, prev_std_val)

    # Calculate RMSE, MAPE, MAE
    rmse = get_rmse(y_test, est)
    mape = get_mape(y_test, est)
    mae = get_mae(y_test, est)
    
    return rmse, mape, mae, est, model.feature_importances_

def add_lags(df, N, lag_cols):
    """
    Add lags up to N number of days to use as features
    The lag columns are labelled as 'adj_close_lag_1', 'adj_close_lag_2', ... etc.
    """
    # Use lags up to N number of days to use as features
    df_w_lags = df.copy()
    df_w_lags.loc[:, 'order_day'] = [x for x in list(range(len(df)))] # Add a column 'order_day' to indicate the order of the rows by date
    merging_keys = ['order_day'] # merging_keys
    shift_range = [x+1 for x in range(N)]
    for shift in shift_range:
        train_shift = df_w_lags[merging_keys + lag_cols].copy()
    
        # E.g. order_day of 0 becomes 1, for shift = 1.
        # So when this is merged with order_day of 1 in df_w_lags, this will represent lag of 1.
        train_shift['order_day'] = train_shift['order_day'] + shift
    
        foo = lambda x: '{}_lag_{}'.format(x, shift) if x in lag_cols else x
        train_shift = train_shift.rename(columns=foo)

        df_w_lags = pd.merge(df_w_lags, train_shift, on=merging_keys, how='left') #.fillna(0)
    del train_shift
    
    return df_w_lags

def get_error_metrics(df,
                      train_size,
                      N,
                      H,
                      seed=100,
                      n_estimators=100,
                      max_depth=3,
                      learning_rate=0.1,
                      min_child_weight=1,
                      subsample=1,
                      colsample_bytree=1,
                      colsample_bylevel=1,
                      gamma=0):
    """
    Given a series consisting of both train+validation, do predictions of forecast horizon H on the validation set, 
    at H/2 intervals.
    Inputs
        df                 : train + val dataframe. len(df) = train_size + val_size
        train_size         : size of train set
        N                  : for feature at day t, we use lags from t-1, t-2, ..., t-N as features
        H                  : forecast horizon
        seed               : model seed
        n_estimators       : number of boosted trees to fit
        max_depth          : maximum tree depth for base learners
        learning_rate      : boosting learning rate (xgb’s “eta”)
        min_child_weight   : minimum sum of instance weight(hessian) needed in a child
        subsample          : subsample ratio of the training instance
        colsample_bytree   : subsample ratio of columns when constructing each tree
        colsample_bylevel  : subsample ratio of columns for each split, in each level
        gamma              : 

    Outputs
        mean of rmse, mean of mape, mean of mae, dictionary of predictions
    """
    rmse_list = [] # root mean square error
    mape_list = [] # mean absolute percentage error
    mae_list = []  # mean absolute error
    preds_dict = {}
    
    # Add lags up to N number of days to use as features
    df = add_lags(df, N, ['Close'])
    
    # Get mean and std dev at timestamp t using values from t-1, ..., t-N
    df = get_mov_avg_std(df, 'Close', N)
    
    # Do scaling
    df = do_scaling(df, N)
    
    # Get list of features
    features = []
    for n in range(N,0,-1):
        features.append("Close_scaled_lag_"+str(n))
    
    for i in range(train_size, len(df)-H+1, int(H/2)):
        # Split into train and test
        train = df[i-train_size:i].copy()
        test = df[i:i+H].copy()
    
        # Drop the NaNs in train
        train.dropna(axis=0, how='any', inplace=True)
    
        # Split into X and y
        X_train_scaled = train[features]
        y_train_scaled = train['Close_scaled']
        y_test = test['Close']
        prev_vals = train[-N:]['Close'].to_numpy()
        prev_mean_val = test.iloc[0]['Close_mean']
        prev_std_val = test.iloc[0]['Close_std']
            
        rmse, mape, mae, est, _ = train_pred_eval_model(X_train_scaled,
                                                        y_train_scaled,
                                                        y_test,
                                                        N,
                                                        H,
                                                        prev_vals,
                                                        prev_mean_val,
                                                        prev_std_val,
                                                        seed=seed,
                                                        n_estimators=n_estimators,
                                                        max_depth=max_depth,
                                                        learning_rate=learning_rate,
                                                        min_child_weight=min_child_weight,
                                                        subsample=subsample,
                                                        colsample_bytree=colsample_bytree,
                                                        colsample_bylevel=colsample_bylevel,
                                                        gamma=gamma)
#         print("N = " + str(N) + ", i = " + str(i) + ", rmse = " + str(rmse) + ", mape = " + str(mape) + ", mae = " + str(mae))
        
        rmse_list.append(rmse)
        mape_list.append(mape)
        mae_list.append(mae)
        preds_dict[i] = est
    
    return np.mean(rmse_list), np.mean(mape_list), np.mean(mae_list), preds_dict 

def get_error_metrics_one_pred(df,
                               train_size,
                               N,
                               H,
                               seed=100,
                               n_estimators=100,
                               max_depth=3,
                               learning_rate=0.1,
                               min_child_weight=1,
                               subsample=1,
                               colsample_bytree=1,
                               colsample_bylevel=1,
                               gamma=0):
    """
    Given a series consisting of both train+test, do one prediction of forecast horizon H on the test set.
    Inputs
        df                 : train + test dataframe. len(df) = train_size + test_size
        train_size         : size of train set
        N                  : for feature at day t, we use lags from t-1, t-2, ..., t-N as features
        H                  : forecast horizon
        seed               : model seed
        n_estimators       : number of boosted trees to fit
        max_depth          : maximum tree depth for base learners
        learning_rate      : boosting learning rate (xgb’s “eta”)
        min_child_weight   : minimum sum of instance weight(hessian) needed in a child
        subsample          : subsample ratio of the training instance
        colsample_bytree   : subsample ratio of columns when constructing each tree
        colsample_bylevel  : subsample ratio of columns for each split, in each level
        gamma              : 

    Outputs
        rmse, mape, mae, predictions
    """    
    # Add lags up to N number of days to use as features
    df = add_lags(df, N, ['Close'])
    
    # Get mean and std dev at timestamp t using values from t-1, ..., t-N
    df = get_mov_avg_std(df, 'Close', N)
    
    # Do scaling
    df = do_scaling(df, N)
    
    # Get list of features
    features = [] # features contain all features, including adj_close_lags
    for n in range(N,0,-1):
        features.append("Close_scaled_lag_"+str(n))
    
    # Split into train and test
    train = df[:train_size].copy()
    test = df[train_size:train_size+H].copy()
    
    # Drop the NaNs in train
    train.dropna(axis=0, how='any', inplace=True)
    
    # Split into X and y
    X_train_scaled = train[features]
    y_train_scaled = train['Close_scaled']
    y_test = test['Close']
    prev_vals = train[-N:]['Close'].to_numpy()
    prev_mean_val = test.iloc[0]['Close_mean']
    prev_std_val = test.iloc[0]['Close_std']
            
    rmse, mape, mae, est, feature_importances = train_pred_eval_model(X_train_scaled,
                                                                      y_train_scaled,
                                                                      y_test,
                                                                      N,
                                                                      H,
                                                                      prev_vals,
                                                                      prev_mean_val,
                                                                      prev_std_val,
                                                                      seed=seed,
                                                                      n_estimators=n_estimators,
                                                                      max_depth=max_depth,
                                                                      learning_rate=learning_rate,
                                                                      min_child_weight=min_child_weight,
                                                                      subsample=subsample,
                                                                      colsample_bytree=colsample_bytree,
                                                                      colsample_bylevel=colsample_bylevel,
                                                                      gamma=gamma)
    
    return rmse, mape, mae, est, feature_importances, features

In [19]:
get_rmse([2],[3])

1.0

In [21]:
import joblib

def fit_save_model(df,
                   train_size,
                   N,
                   H,
                   outpath,
                   seed=100,
                   n_estimators=100,
                   max_depth=3,
                   learning_rate=0.1,
                   min_child_weight=1,
                   subsample=1,
                   colsample_bytree=1,
                   colsample_bylevel=1,
                   gamma=0):
    """
    Fit and save model
    Inputs
        df                 : train + test dataframe. len(df) = train_size + test_size
        train_size         : size of train set
        N                  : for feature at day t, we use lags from t-1, t-2, ..., t-N as features
        H                  : forecast horizon
        outpath            : 
        seed               : model seed
        n_estimators       : number of boosted trees to fit
        max_depth          : maximum tree depth for base learners
        learning_rate      : boosting learning rate (xgb’s “eta”)
        min_child_weight   : minimum sum of instance weight(hessian) needed in a child
        subsample          : subsample ratio of the training instance
        colsample_bytree   : subsample ratio of columns when constructing each tree
        colsample_bylevel  : subsample ratio of columns for each split, in each level
        gamma              : 

    Outputs
        rmse, mape, mae, predictions
    """    
    # Add lags up to N number of days to use as features
    df = add_lags(df, N, ['Close'])
    
    # Get mean and std dev at timestamp t using values from t-1, ..., t-N
    df = get_mov_avg_std(df, 'Close', N)
    
    # Do scaling
    df = do_scaling(df, N)
    
    # Get list of features
    features = [] # features contain all features, including adj_close_lags
    for n in range(N,0,-1):
        features.append("Close_scaled_lag_"+str(n))
    
    # Split into train and test
    train = df[:train_size].copy()
    test = df[train_size:train_size+H].copy()
    
    # Drop the NaNs in train
    #train.dropna(axis=0, how='any', inplace=True)
    
    # Split into X and y
    X_train_scaled = train[features]
    y_train_scaled = train['Close_scaled']
   
    # Generate model
    model = XGBRegressor(objective ='reg:squarederror',
                         seed=100,
                         n_estimators=n_estimators,
                         max_depth=max_depth,
                         learning_rate=learning_rate,
                         min_child_weight=min_child_weight,
                         subsample=subsample,
                         colsample_bytree=colsample_bytree,
                         colsample_bylevel=colsample_bylevel,
                         gamma=gamma)
    
    # Train the model
    model.fit(X_train_scaled, y_train_scaled)
    
            
    joblib.dump(model, outpath)
# print(fit_save_model(df,
#                    10,
#                    3,
#                    5,
#                    "outpath",
#                    seed=100,
#                    n_estimators=100,
#                    max_depth=3,
#                    learning_rate=0.1,
#                    min_child_weight=1,
#                    subsample=1,
#                    colsample_bytree=1,
#                    colsample_bylevel=1,
#                    gamma=0))

In [22]:
df.head()

,Date,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover
5286,2021-03-31,1552.20,1549.9,1567.4,1536.75,1556.95,1558.05,1553.18,1431746,2.223752e+14
5285,2021-03-30,1505.85,1510.0,1563.0,1510.00,1549.95,1552.20,1551.26,3466049,5.376747e+14
5284,2021-03-26,1450.85,1459.9,1512.4,1457.00,1505.50,1505.85,1490.91,1922278,2.865948e+14
5283,2021-03-25,1488.15,1484.1,1498.5,1447.75,1451.60,1450.85,1467.78,1609996,2.363123e+14
5282,2021-03-24,1495.35,1486.0,1508.8,1476.45,1485.00,1488.15,1492.58,1871092,2.792753e+14


In [21]:
# #df.drop(["prev_close","open","high","low","last","vwap","volume","turnover"],axis = 1, inplace = True)
# #df.columns= ["Date","Close"]
# df.columns = ["Date","Close","Month"]
# df.drop(["Month"],axis = 1, inplace = True)
# df.head()

In [25]:
df.head()
df.drop(["prev_close","open","high","low","last","vwap","volume","turnover"],axis = 1, inplace = True)
df.columns= ["Date","Close","Month"]
# df.columns = ["Date","Close","Month"]
# df.drop(["Month"],axis = 1, inplace = True)

In [24]:
df
pred_day = 1470                # Predict for this day, for the next H-1 days. Note indexing of days start from 0.

H = 21                         # Forecast horizon, in days. Note there are about 252 trading days in a year
train_size = 252*3             # Use 3 years of data as train set. Note there are about 252 trading days in a year
val_size = 252                 # Use 1 year of data as validation set
N = 10                          # for feature at day t, we use lags from t-1, t-2, ..., t-N as features

n_estimators = 100             # Number of boosted trees to fit. default = 100
max_depth = 3                  # Maximum tree depth for base learners. default = 3
learning_rate = 0.1            # Boosting learning rate (xgb’s “eta”). default = 0.1
min_child_weight = 1           # Minimum sum of instance weight(hessian) needed in a child. default = 1
subsample = 1                  # Subsample ratio of the training instance. default = 1
colsample_bytree = 1           # Subsample ratio of columns when constructing each tree. default = 1
colsample_bylevel = 1          # Subsample ratio of columns for each split, in each level. default = 1
gamma = 0                      # Minimum loss reduction required to make a further partition on a leaf node of the tree. default=0

model_seed = 100

fontsize = 14
ticklabelsize = 14

# Plotly colors
colors = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
]

####################################

train_val_size = train_size + val_size # Size of train+validation set
print("No. of days in train+validation set = " + str(train_val_size))

df.loc[:, 'Date'] = pd.to_datetime(df['Date'],format='%Y-%m-%d')
df.columns = [str(x).lower().replace(' ', '_') for x in df.columns]
df['month'] = df['date'].dt.month
df.sort_values(by='date', inplace=True, ascending=True)

print("Predicting on day %d, date %s, with forecast horizon H = %d" % (pred_day, df.iloc[pred_day]['date'], H))
df.head()

No. of days in train+validation set = 1008
Predicting on day 1470, date 2005-11-09 00:00:00, with forecast horizon H = 21


,date,prev_close,open,high,low,last,close,vwap,volume,turnover,month
0,2000-01-03,144.95,146.0,156.45,146.0,155.00,155.70,154.36,23000,3.550370e+11,1
1,2000-01-04,155.70,152.0,152.00,144.0,148.05,147.40,146.72,26300,3.858700e+11,1
2,2000-01-05,147.40,144.0,148.80,136.0,139.95,138.40,142.95,20600,2.944850e+11,1
3,2000-01-06,138.40,142.0,149.50,141.0,149.50,149.50,147.87,31600,4.672750e+11,1
4,2000-01-07,149.50,149.5,153.00,145.0,147.75,146.35,148.63,36600,5.439965e+11,1


In [63]:

#df.drop(["Month"],axis = 1,inplace = True)
df.head()


,Date,Close,Month
0,2000-01-03,155.70,1
1,2000-01-04,147.40,1
2,2000-01-05,138.40,1
3,2000-01-06,149.50,1
4,2000-01-07,146.35,1


In [26]:
train = df[pred_day-train_val_size:pred_day-val_size].copy()
val = df[pred_day-val_size:pred_day].copy()
train_val = df[pred_day-train_val_size:pred_day].copy()
test = df[pred_day:pred_day+H].copy()
print("train.shape = " + str(train.shape))
print("val.shape = " + str(val.shape))
print("train_val.shape = " + str(train_val.shape))
print("test.shape = " + str(test.shape))
df.head()

train.shape = (756, 3)
val.shape = (252, 3)
train_val.shape = (1008, 3)
test.shape = (21, 3)


,Date,Close,Month
0,2000-01-03,155.70,1
1,2000-01-04,147.40,1
2,2000-01-05,138.40,1
3,2000-01-06,149.50,1
4,2000-01-07,146.35,1


In [ ]:
df.columns = ["Date","Close"]
df.head()

In [ ]:
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=train['Date'], 
                         y=train['Close'],
                         mode='lines',
                         name='train',
                         line=dict(color='blue')))
fig.add_trace(go.Scatter(x=val['Date'], 
                         y=val['Close'],
                         mode='lines',
                         name='validation',
                         line=dict(color='orange')))
fig.add_trace(go.Scatter(x=test['Date'], 
                         y=test['Close'],
                         mode='lines',
                         name='test',
                         line=dict(color='green')))

# Plot the predictions
n = 0
for key in preds_dict:
    fig.add_trace(go.Scatter(x=train_val[key:key+H]['Date'], 
                             y=preds_dict[key],
                             mode='lines',
                             name='predictions',
                             line=dict(color=colors[n%len(colors)])))
    n = n + 1

fig.update_layout(yaxis=dict(title='USD'),
                  xaxis=dict(title='Date'))

In [ ]:
imp = list(zip(features, feature_importances))
imp.sort(key=lambda tup: tup[1]) 
imp